In [1]:
import pickle
import pandas as pd
import re

In [2]:
import time
import datetime
import seaborn as sns
import numpy as np

In [3]:
def replace_logical(s):
    if s:
        return 1
    else:
        return 0
def transformYear(val):
    #year = datetime.datetime.fromtimestamp(int(df_train.date_created[0])).year
    year = datetime.datetime.fromtimestamp(int(val)).year
    return year

def transformMonth(val):
    month = datetime.datetime.fromtimestamp(val).month
    return month

def get_latitude(val):
    res = val['latitude']
    return res

def get_longitude(val):
    res = val['longitude']
    return res

def preprocess_data(df_input):
    df_output = df_input
    df_output = pd.get_dummies(df_output)
    return df_output

def data_initial_preproc (df_input):
    
    df_output = df_input
    df_output['price'] = df_output['price']/100
    df_output['can_buy'] = df_output['can_buy'].map(lambda s: replace_logical(s))
    df_output['can_promote'] = df_output['can_promote'].map(lambda s: replace_logical(s))
    df_output['contacts_visible'] = df_output['contacts_visible'].map(lambda s: replace_logical(s))
    df_output['mortgage_available'] = df_output['mortgage_available'].map(lambda s: replace_logical(s))
    df_output['delivery_available'] = df_output['delivery_available'].map(lambda s: replace_logical(s))
    df_output['payment_available'] = df_output['payment_available'].map(lambda s: replace_logical(s))
    df_output['year']=df_output['date_created'].map(transformYear)
    df_output['month']=df_output['date_created'].map(transformMonth)
    df_output = df_output.drop(['date_created', 'description', 'fields', 'id', 'images', 'subway'], axis = 1)
    df_output = df_output.drop(['can_buy', 'can_promote', 'category'], axis = 1)
    df_output = df_output[df_output['price']>0]
    df_output = df_output.drop(['name'], axis = 1)
    df_output['latitude'] = df_output['location'].map(get_latitude)
    df_output['longitude'] = df_output['location'].map(get_longitude)
    df_output = df_output.drop(['location'],axis = 1)
    df_output['price'] = np.log10(df_output['price'] + 1)
    df_output['subcategory'] = df_output['subcategory'].map(lambda s: 'S' + str(s))
    df_output = df_output.pipe(preprocess_data)
    
    return df_output

In [4]:
df_train =pickle.load(open("./train_sample.pckl",'rb'))

In [5]:
df_train = df_train.pipe(data_initial_preproc)

In [7]:
df_train.head(10)

,contacts_visible,delivery_available,mortgage_available,payment_available,price,year,month,latitude,longitude,subcategory_S10001,...,subcategory_S907,subcategory_S908,subcategory_S909,subcategory_S910,subcategory_S911,subcategory_S912,subcategory_S914,subcategory_S919,subcategory_S920,subcategory_S921
0,1,0,0,1,7.600902,2017,4,55.806888,37.546077,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,5.860786,2016,10,55.692979,37.872337,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,5.707110,2016,9,55.639011,37.349378,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,6.685861,2016,10,55.847334,37.495834,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,1,6.216606,2017,8,60.044826,30.355460,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,5.707110,2017,6,56.367960,43.818292,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,7.313887,2017,3,51.638327,39.267098,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,1,7.601402,2017,4,45.017036,41.923240,0,...,0,0,0,0,0,0,0,0,0,0
10,1,0,0,0,5.860786,2016,12,53.279983,50.273564,0,...,0,0,0,0,0,0,0,0,0,0
11,1,0,0,1,8.160804,2016,11,53.727380,84.926507,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_train.shape

(97193, 257)

In [9]:
X = df_train.drop(['price'],axis=1).values
y = df_train['price'].values

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [12]:
from sklearn.metrics import mean_squared_error

In [20]:
#from sklearn.linear_model import LinearRegression
#model = LinearRegression(n_jobs = 1000, normalize = True)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#result = mean_squared_error(y_test, y_pred)
#result

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50, n_jobs=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
result = mean_squared_error(y_test, y_pred)

In [ ]:
result